# Analyzing Hurricane Elsa with 311 Street Flooding Complaints in NYC

In [142]:
# importing libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
import os

In [143]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [144]:
%watermark -v -p numpy,pandas

CPython 3.7.1
IPython 7.20.0

numpy 1.19.2
pandas 1.2.1


In [145]:
socrata_domain = 'data.cityofnewyork.us'
socrata_dataset_identifier = 'erm2-nwe9'

# sodapy key
socrata_token = os.environ.get("SODAPY_APPTOKEN")

In [146]:
client = Socrata(socrata_domain, socrata_token)

# good practice to review the columns
metadata = client.get_metadata(socrata_dataset_identifier)
[x['fieldName'] for x in metadata['columns']]

['unique_key',
 'created_date',
 'closed_date',
 'agency',
 'agency_name',
 'complaint_type',
 'descriptor',
 'location_type',
 'incident_zip',
 'incident_address',
 'street_name',
 'cross_street_1',
 'cross_street_2',
 'intersection_street_1',
 'intersection_street_2',
 'address_type',
 'city',
 'landmark',
 'facility_type',
 'status',
 'due_date',
 'resolution_description',
 'resolution_action_updated_date',
 'community_board',
 'bbl',
 'borough',
 'x_coordinate_state_plane',
 'y_coordinate_state_plane',
 'open_data_channel_type',
 'park_facility_name',
 'park_borough',
 'vehicle_type',
 'taxi_company_borough',
 'taxi_pick_up_location',
 'bridge_highway_name',
 'bridge_highway_direction',
 'road_ramp',
 'bridge_highway_segment',
 'latitude',
 'longitude',
 'location',
 ':@computed_region_efsh_h5xi',
 ':@computed_region_f5dn_yrer',
 ':@computed_region_yeji_bk3q',
 ':@computed_region_92fq_4b7q',
 ':@computed_region_sbqj_enih']

# exploring the dataset

In [147]:
client = Socrata("data.cityofnewyork.us", socrata_token, timeout=1000)

query = """
SELECT 
    descriptor, count(descriptor)
WHERE 
    LOWER(descriptor) LIKE '%flood%'
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
"""

results = client.get("erm2-nwe9", query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print(results_df.shape)
results_df

(11, 2)


,descriptor,count_descriptor
0,Catch Basin Clogged/Flooding (Use Comments) (SC),92386
1,Street Flooding (SJ),28688
2,Flood Light Lamp Out,6041
3,Highway Flooding (SH),2869
4,Flood Light Lamp Cycling,2521
5,Ready NY - Flooding,271
6,Flood Light Lamp Dayburning,207
7,Flood Light Lamp Missing,193
8,Flood Light Lamp Dim,178
9,RAIN GARDEN FLOODING (SRGFLD),105


In [148]:
client = Socrata("data.cityofnewyork.us", socrata_token, timeout=1000)

query = """
SELECT 
    descriptor, count(descriptor)
WHERE 
    complaint_type='Sewer'
GROUP BY 
    descriptor
ORDER BY 
    count(descriptor) DESC
"""

results = client.get("erm2-nwe9", query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print(results_df.shape)
results_df.head(len(results_df))

(27, 2)


,descriptor,count_descriptor
0,Sewer Backup (Use Comments) (SA),154184
1,Catch Basin Clogged/Flooding (Use Comments) (SC),92386
2,Catch Basin Sunken/Damaged/Raised (SC1),29237
3,Street Flooding (SJ),28688
4,Manhole Cover Broken/Making Noise (SB),19793
5,Manhole Cover Missing (Emergency) (SA3),18067
6,Sewer Odor (SA2),15921
7,Defective/Missing Curb Piece (SC4),8640
8,Manhole Overflow (Use Comments) (SA1),7170
9,Catch Basin Search (SC2),4154


# query data for street flooding complaints

In [149]:
client = Socrata("data.cityofnewyork.us", socrata_token, timeout=1000)

query = """
SELECT 
    *
WHERE 
    descriptor = 'Street Flooding (SJ)'
LIMIT 
    1000000
"""

results = client.get("erm2-nwe9", query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print(results_df.shape)
results_df.head()

(28688, 32)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,...,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,intersection_street_1,intersection_street_2,facility_type,due_date
0,50465779,2021-05-04T17:58:00.000,2021-05-05T07:15:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11226,2225 TILDEN AVENUE,TILDEN AVENUE,...,PHONE,Unspecified,BROOKLYN,40.64660766955987,-73.95629594944009,"{'latitude': '40.64660766955987', 'longitude':...",NaN,NaN,NaN,NaN
1,50471923,2021-05-04T10:25:00.000,2021-05-05T08:45:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11234,1477 EAST 51 STREET,EAST 51 STREET,...,ONLINE,Unspecified,BROOKLYN,40.62449690722713,-73.92644675494573,"{'latitude': '40.62449690722713', 'longitude':...",NaN,NaN,NaN,NaN
2,45123962,2019-12-10T12:13:00.000,2019-12-11T11:45:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11434,NaN,NaN,...,PHONE,Unspecified,QUEENS,40.66016771824149,-73.77422648953312,"{'latitude': '40.66016771824149', 'longitude':...",147 AVENUE,ROCKAWAY BOULEVARD,NaN,NaN
3,45123981,2019-12-10T16:23:00.000,2019-12-11T08:30:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10022,NaN,NaN,...,PHONE,Unspecified,MANHATTAN,40.761668277085114,-73.97491556118797,"{'latitude': '40.761668277085114', 'longitude'...",5 AVENUE,WEST 55 STREET,NaN,NaN
4,45123982,2019-12-10T14:30:00.000,2019-12-15T09:20:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11205,NaN,NaN,...,PHONE,Unspecified,BROOKLYN,40.69909044367837,-73.95607401194755,"{'latitude': '40.69909044367837', 'longitude':...",FLUSHING AVENUE,SPENCER STREET,NaN,NaN


In [150]:
print('Number of total records: {:,}.'.format(len(results_df)))
print('Min: {}.'.format(results_df.created_date.min()))
print('Max: {}'.format(results_df.created_date.max()))

Number of total records: 28,688.
Min: 2010-01-02T08:26:00.000.
Max: 2021-07-10T21:02:00.000


In [151]:
# saving only data for hurricane elsa
elsa_df = results_df.loc[(results_df["created_date"] > '2021-07-07') & 
                         (results_df["created_date"] < '2021-07-13')]

print(elsa_df.shape)
elsa_df.head()

(166, 32)


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,...,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,intersection_street_1,intersection_street_2,facility_type,due_date
75,51143675,2021-07-08T16:04:00.000,2021-07-09T10:30:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10471,6485 BROADWAY,BROADWAY,...,ONLINE,Unspecified,BRONX,40.905351378307486,-73.89645716668805,"{'latitude': '40.905351378307486', 'longitude'...",NaN,NaN,NaN,NaN
76,51145094,2021-07-08T16:00:00.000,2021-07-08T20:45:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10034,200 NAGLE AVENUE,NAGLE AVENUE,...,ONLINE,Unspecified,MANHATTAN,40.86210236351487,-73.9233713087911,"{'latitude': '40.86210236351487', 'longitude':...",NaN,NaN,NaN,NaN
79,51146482,2021-07-08T21:04:00.000,NaN,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11236,1035 EAST 102 STREET,EAST 102 STREET,...,ONLINE,Unspecified,BROOKLYN,40.6437429889248,-73.89427228300471,"{'latitude': '40.6437429889248', 'longitude': ...",NaN,NaN,NaN,NaN
84,51148924,2021-07-08T17:04:00.000,2021-07-09T07:44:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10471,NaN,NaN,...,ONLINE,Unspecified,BRONX,40.900731405929164,-73.901969964742,"{'latitude': '40.900731405929164', 'longitude'...",FIELDSTON ROAD,WEST 253 STREET,NaN,NaN
89,51139519,2021-07-08T16:44:00.000,2021-07-09T17:15:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11432,172-39 HIGHLAND AVENUE,HIGHLAND AVENUE,...,PHONE,Unspecified,QUEENS,40.71338015366497,-73.78984933362548,"{'latitude': '40.71338015366497', 'longitude':...",NaN,NaN,NaN,NaN


In [152]:
print('Number of total records: {:,}.'.format(len(elsa_df)))      
print('Min: {}.'.format(elsa_df.created_date.min()))
print('Max: {}'.format(elsa_df.created_date.max()))

Number of total records: 166.
Min: 2021-07-07T00:10:00.000.
Max: 2021-07-10T21:02:00.000


In [153]:
print('number of complaints not closed: {}.'.format(elsa_df['closed_date'].isnull().sum()))
print('% of total: {:.2f}%.'.format(elsa_df['closed_date'].isnull().sum() / len(elsa_df) * 100))

number of complaints not closed: 92.
% of total: 55.42%.


In [154]:
# exporting data as csv
elsa_df.to_csv('elsa_311_calls.csv', index=False)

In [155]:
# sanity check
%ls

README.md             elsa_311_calls.csv    query_tutorial.ipynb


In [156]:
# sanity check
test = pd.read_csv('elsa_311_calls.csv')

print('Number of total records: {:,}.'.format(len(test)))      
print('Min: {}.'.format(test.created_date.min()))
print('Max: {}'.format(test.created_date.max()))

test.head()

Number of total records: 166.
Min: 2021-07-07T00:10:00.000.
Max: 2021-07-10T21:02:00.000


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,incident_address,street_name,...,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,intersection_street_1,intersection_street_2,facility_type,due_date
0,51143675,2021-07-08T16:04:00.000,2021-07-09T10:30:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10471,6485 BROADWAY,BROADWAY,...,ONLINE,Unspecified,BRONX,40.905351,-73.896457,"{'latitude': '40.905351378307486', 'longitude'...",NaN,NaN,NaN,NaN
1,51145094,2021-07-08T16:00:00.000,2021-07-08T20:45:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10034,200 NAGLE AVENUE,NAGLE AVENUE,...,ONLINE,Unspecified,MANHATTAN,40.862102,-73.923371,"{'latitude': '40.86210236351487', 'longitude':...",NaN,NaN,NaN,NaN
2,51146482,2021-07-08T21:04:00.000,NaN,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11236,1035 EAST 102 STREET,EAST 102 STREET,...,ONLINE,Unspecified,BROOKLYN,40.643743,-73.894272,"{'latitude': '40.6437429889248', 'longitude': ...",NaN,NaN,NaN,NaN
3,51148924,2021-07-08T17:04:00.000,2021-07-09T07:44:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),10471,NaN,NaN,...,ONLINE,Unspecified,BRONX,40.900731,-73.901970,"{'latitude': '40.900731405929164', 'longitude'...",FIELDSTON ROAD,WEST 253 STREET,NaN,NaN
4,51139519,2021-07-08T16:44:00.000,2021-07-09T17:15:00.000,DEP,Department of Environmental Protection,Sewer,Street Flooding (SJ),11432,172-39 HIGHLAND AVENUE,HIGHLAND AVENUE,...,PHONE,Unspecified,QUEENS,40.713380,-73.789849,"{'latitude': '40.71338015366497', 'longitude':...",NaN,NaN,NaN,NaN
